In [166]:
import pandas as pd
from multielo import MultiElo
from collections import namedtuple

STARTING_ELO = 1200
ELO_DICT = {}
RACES = []
DRIVERS = []

class Circuit:
    def __init__(self, circuitId, reference, name, location, country, wiki):
        self.id = circuitId
        self.reference = reference
        self.name = name
        self.location = location
        self.country = country
        self.wiki = wiki

    def __str__(self):
        return f'<Circuit {self.id}>'
    
    
class Race:
    def __init__(self, raceId, year, circuitId, name, date, wiki):
        self.id = raceId
        self.year = year
        self.circuitId = circuitId
        self.name = name
        self.date = date
        self.wiki = wiki
        self.drivers = []
        self.results = []
        #self.excluded = []
        
    def __str__(self):
        return f'<Race {self.id}>'

    
class Driver:
    def __init__(self, driverId, reference, number, code, firstName, lastName, dob, nationality, wiki):
        self.id = driverId
        self.reference = reference
        self.number = number
        self.code = code
        self.firstName = firstName
        self.lastName = lastName
        self.dob = dob
        self.nationality = nationality
        self.wiki = wiki
        self.elo = STARTING_ELO
        self.ratingChanges = {}
    
    def __str__(self):
        return f'<Driver {self.id}>'
    
    def ChangeRating(self, race, elo):
        self.ratingChanges[race] = (self.elo, elo)
        self.elo = elo
    
    
def DriverRankingDelta(driver, race):
    if (driver.id not in race.drivers):
        raise ValueError("Driver did not participate in race.")
    else:
        return driver.ratingChanges[race.id][1] - driver.ratingChanges[race.id][0]

In [168]:
# Driver data loading
df_drivers = pd.read_csv('data/drivers.csv')
drivers = []
for index, row in df_drivers.iterrows():
    driver = Driver(row['driverId'], row['driverRef'], row['number'], row['code'], row['forename'], row['surname'], row['dob'], row['nationality'], row['url'])
    drivers.append(driver)           

for driver in drivers:
    ELO_DICT[driver.id] = 1200
DRIVERS = drivers

1


In [169]:
# Race data loading
df_races = pd.read_csv('data/races.csv')
races = []
for index, row in df_races.iterrows():
    race = Race(row['raceId'], row['year'], row['circuitId'], row['name'], row['date'], row['url'])
    races.append(race)

RACES = races

<Race 1>


In [170]:
# Circuit data loading
df_circuits = pd.read_csv('data/circuits.csv')
circuits = []
for index, row in df_circuits.iterrows():
    circuit = Circuit(row['circuitId'], row['circuitRef'], row['name'], row['location'], row['country'], row['url'])
    circuits.append(circuit)
print(circuits[0])

<Circuit 1>


In [171]:
# Result data loading
df_results = pd.read_csv('data/results.csv')
df_results.head()

1073


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [173]:
for race in RACES:
    id = race.id
    df = df_results.loc[df_results['raceId'] == id].sort_values(by='positionOrder')
    drivers = []
    for index, row in df.iterrows():
        drivers.append((int(row['driverId']),int(row['positionOrder'])))

    drivers.sort(key=lambda x: x[1])

    for d,r in drivers:
        race.drivers.append(d)
        race.results.append(r)

Australian Grand Prix 2009 (1)
Malaysian Grand Prix 2009 (2)
Chinese Grand Prix 2009 (3)
Bahrain Grand Prix 2009 (4)
Spanish Grand Prix 2009 (5)
Monaco Grand Prix 2009 (6)
Turkish Grand Prix 2009 (7)
British Grand Prix 2009 (8)
German Grand Prix 2009 (9)
Hungarian Grand Prix 2009 (10)
European Grand Prix 2009 (11)
Belgian Grand Prix 2009 (12)
Italian Grand Prix 2009 (13)
Singapore Grand Prix 2009 (14)
Japanese Grand Prix 2009 (15)
Brazilian Grand Prix 2009 (16)
Abu Dhabi Grand Prix 2009 (17)
Australian Grand Prix 2008 (18)
Malaysian Grand Prix 2008 (19)
Bahrain Grand Prix 2008 (20)
Spanish Grand Prix 2008 (21)
Turkish Grand Prix 2008 (22)
Monaco Grand Prix 2008 (23)
Canadian Grand Prix 2008 (24)
French Grand Prix 2008 (25)
British Grand Prix 2008 (26)
German Grand Prix 2008 (27)
Hungarian Grand Prix 2008 (28)
European Grand Prix 2008 (29)
Belgian Grand Prix 2008 (30)
Italian Grand Prix 2008 (31)
Singapore Grand Prix 2008 (32)
Japanese Grand Prix 2008 (33)
Chinese Grand Prix 2008 (34)
B

Canadian Grand Prix 1993 (279)
French Grand Prix 1993 (280)
British Grand Prix 1993 (281)
German Grand Prix 1993 (282)
Hungarian Grand Prix 1993 (283)
Belgian Grand Prix 1993 (284)
Italian Grand Prix 1993 (285)
Portuguese Grand Prix 1993 (286)
Japanese Grand Prix 1993 (287)
Australian Grand Prix 1993 (288)
South African Grand Prix 1992 (289)
Mexican Grand Prix 1992 (290)
Brazilian Grand Prix 1992 (291)
Spanish Grand Prix 1992 (292)
San Marino Grand Prix 1992 (293)
Monaco Grand Prix 1992 (294)
Canadian Grand Prix 1992 (295)
French Grand Prix 1992 (296)
British Grand Prix 1992 (297)
German Grand Prix 1992 (298)
Hungarian Grand Prix 1992 (299)
Belgian Grand Prix 1992 (300)
Italian Grand Prix 1992 (301)
Portuguese Grand Prix 1992 (302)
Japanese Grand Prix 1992 (303)
Australian Grand Prix 1992 (304)
United States Grand Prix 1991 (305)
Brazilian Grand Prix 1991 (306)
San Marino Grand Prix 1991 (307)
Monaco Grand Prix 1991 (308)
Canadian Grand Prix 1991 (309)
Mexican Grand Prix 1991 (310)
Fre

French Grand Prix 1974 (598)
British Grand Prix 1974 (599)
German Grand Prix 1974 (600)
Austrian Grand Prix 1974 (601)
Italian Grand Prix 1974 (602)
Canadian Grand Prix 1974 (603)
United States Grand Prix 1974 (604)
Argentine Grand Prix 1973 (605)
Brazilian Grand Prix 1973 (606)
South African Grand Prix 1973 (607)
Spanish Grand Prix 1973 (608)
Belgian Grand Prix 1973 (609)
Monaco Grand Prix 1973 (610)
Swedish Grand Prix 1973 (611)
French Grand Prix 1973 (612)
British Grand Prix 1973 (613)
Dutch Grand Prix 1973 (614)
German Grand Prix 1973 (615)
Austrian Grand Prix 1973 (616)
Italian Grand Prix 1973 (617)
Canadian Grand Prix 1973 (618)
United States Grand Prix 1973 (619)
Argentine Grand Prix 1972 (620)
South African Grand Prix 1972 (621)
Spanish Grand Prix 1972 (622)
Monaco Grand Prix 1972 (623)
Belgian Grand Prix 1972 (624)
French Grand Prix 1972 (625)
British Grand Prix 1972 (626)
German Grand Prix 1972 (627)
Austrian Grand Prix 1972 (628)
Italian Grand Prix 1972 (629)
Canadian Grand 

Hungarian Grand Prix 2013 (890)
Belgian Grand Prix 2013 (891)
Italian Grand Prix 2013 (892)
Singapore Grand Prix 2013 (893)
Korean Grand Prix 2013 (894)
Japanese Grand Prix 2013 (895)
Indian Grand Prix 2013 (896)
Abu Dhabi Grand Prix 2013 (897)
United States Grand Prix 2013 (898)
Brazilian Grand Prix 2013 (899)
Australian Grand Prix 2014 (900)
Malaysian Grand Prix 2014 (901)
Bahrain Grand Prix 2014 (902)
Chinese Grand Prix 2014 (903)
Spanish Grand Prix 2014 (904)
Monaco Grand Prix 2014 (905)
Canadian Grand Prix 2014 (906)
Austrian Grand Prix 2014 (907)
British Grand Prix 2014 (908)
German Grand Prix 2014 (909)
Hungarian Grand Prix 2014 (910)
Belgian Grand Prix 2014 (911)
Italian Grand Prix 2014 (912)
Singapore Grand Prix 2014 (913)
Japanese Grand Prix 2014 (914)
Russian Grand Prix 2014 (915)
United States Grand Prix 2014 (916)
Brazilian Grand Prix 2014 (917)
Abu Dhabi Grand Prix 2014 (918)
Monaco Grand Prix 2015 (931)
Canadian Grand Prix 2015 (932)
Bahrain Grand Prix 2015 (929)
Spanish

In [176]:
driver_dict = {}
race_dict = {}
for driver in DRIVERS:
    driver_dict[driver.id] = driver
for race in RACES:
    race_dict[race.id] = race

In [189]:
# reset ELO_DICT
#for driver in DRIVERS:
#    ELO_DICT[driver.id] = 1200
#ELO_DICT

{1: 1200,
 2: 1200,
 3: 1200,
 4: 1200,
 5: 1200,
 6: 1200,
 7: 1200,
 8: 1200,
 9: 1200,
 10: 1200,
 11: 1200,
 12: 1200,
 13: 1200,
 14: 1200,
 15: 1200,
 16: 1200,
 17: 1200,
 18: 1200,
 19: 1200,
 20: 1200,
 21: 1200,
 22: 1200,
 23: 1200,
 24: 1200,
 25: 1200,
 26: 1200,
 27: 1200,
 28: 1200,
 29: 1200,
 30: 1200,
 31: 1200,
 32: 1200,
 33: 1200,
 34: 1200,
 35: 1200,
 36: 1200,
 37: 1200,
 38: 1200,
 39: 1200,
 40: 1200,
 41: 1200,
 42: 1200,
 43: 1200,
 44: 1200,
 45: 1200,
 46: 1200,
 47: 1200,
 48: 1200,
 49: 1200,
 50: 1200,
 51: 1200,
 52: 1200,
 53: 1200,
 54: 1200,
 55: 1200,
 56: 1200,
 57: 1200,
 58: 1200,
 59: 1200,
 60: 1200,
 61: 1200,
 62: 1200,
 63: 1200,
 64: 1200,
 65: 1200,
 66: 1200,
 67: 1200,
 68: 1200,
 69: 1200,
 70: 1200,
 71: 1200,
 72: 1200,
 73: 1200,
 74: 1200,
 75: 1200,
 76: 1200,
 77: 1200,
 78: 1200,
 79: 1200,
 80: 1200,
 81: 1200,
 82: 1200,
 83: 1200,
 84: 1200,
 85: 1200,
 86: 1200,
 87: 1200,
 88: 1200,
 89: 1200,
 90: 1200,
 91: 1200,
 92: 120

In [190]:
elo = MultiElo()
def getResultArray(race):
    arr = []
    for driver in race.drivers:
        arr.append(ELO_DICT[driver])
    return arr
    
for race in races:
    try:
        result = getResultArray(race)
        new = elo.get_new_ratings(result)
        for i,x in enumerate(new):
            ELO_DICT[race.drivers[i]] = x
            driver_dict[race.drivers[i]].ChangeRating(race,x)
    except:
        print(f"Failed Race ID: {race.id}")

Failed Race ID: 1074
Failed Race ID: 1075
Failed Race ID: 1076
Failed Race ID: 1077
Failed Race ID: 1078
Failed Race ID: 1079
Failed Race ID: 1080
Failed Race ID: 1081
Failed Race ID: 1082
Failed Race ID: 1083
Failed Race ID: 1084
Failed Race ID: 1085
Failed Race ID: 1086
Failed Race ID: 1087
Failed Race ID: 1088
Failed Race ID: 1089
Failed Race ID: 1090
Failed Race ID: 1091
Failed Race ID: 1092
Failed Race ID: 1093
Failed Race ID: 1094
Failed Race ID: 1095
Failed Race ID: 1096


In [194]:
ranking = []
for key in ELO_DICT:
    ranking.append((key, ELO_DICT[key]))

ranking.sort(key=lambda x: x[1], reverse=True)
for i,rank in enumerate(ranking):
    pos = i+1
    fname = driver_dict[rank[0]].firstName
    lname = driver_dict[rank[0]].lastName
    elo = rank[1]
    print(f'{pos}. {fname} {lname} ({rank[0]}) - {elo}')

1. Lewis Hamilton (1) - 1736.3119435568615
2. Nico Rosberg (3) - 1669.961404686599
3. Max Verstappen (830) - 1597.6238368811835
4. Valtteri Bottas (822) - 1506.2219426612285
5. Jacques Villeneuve (35) - 1504.810424774383
6. Carlos Sainz (832) - 1484.013124131847
7. Mark Webber (17) - 1434.9747534743585
8. Charles Leclerc (844) - 1432.729986778366
9. Bruce McLaren (360) - 1432.598435513201
10. Phil Hill (403) - 1432.4661526677646
11. Lando Norris (846) - 1431.1119610229591
12. Juan Fangio (579) - 1426.7152536450092
13. Gilles Villeneuve (203) - 1421.2573165295746
14. Sergio Pérez (815) - 1412.4182525057997
15. Nino Farina (642) - 1411.7661661367313
16. Richie Ginther (386) - 1408.8511500503294
17. Alexander Albon (848) - 1405.7735131711565
18. Clay Regazzoni (223) - 1404.8371888336883
19. Patrick Depailler (221) - 1400.4080818961174
20. Jim Clark (373) - 1394.7442639158792
21. Daniel Ricciardo (817) - 1393.0306764159147
22. Mike Hawthorn (578) - 1389.8749437763984
23. David Coulthard (1

In [192]:
# Track elo changes
driver_dict[30].ratingChanges

{<__main__.Race at 0x7fe768e664a8>: (1200, 1228.9037995769368),
 <__main__.Race at 0x7fe768e664e0>: (1228.9037995769368, 1243.7277076537052),
 <__main__.Race at 0x7fe768e66518>: (1243.7277076537052, 1225.3471264930176),
 <__main__.Race at 0x7fe768e66550>: (1225.3471264930176, 1255.0574498647593),
 <__main__.Race at 0x7fe768e66588>: (1255.0574498647593, 1282.4826517024937),
 <__main__.Race at 0x7fe768e665c0>: (1282.4826517024937, 1304.7279135671506),
 <__main__.Race at 0x7fe768e665f8>: (1304.7279135671506, 1316.4796111172316),
 <__main__.Race at 0x7fe768e66630>: (1316.4796111172316, 1336.0499919216234),
 <__main__.Race at 0x7fe768e66668>: (1336.0499919216234, 1354.1788874686374),
 <__main__.Race at 0x7fe768e666a0>: (1354.1788874686374, 1373.8941334164299),
 <__main__.Race at 0x7fe768e666d8>: (1373.8941334164299, 1392.1777216593791),
 <__main__.Race at 0x7fe768e66710>: (1392.1777216593791, 1409.2916344265018),
 <__main__.Race at 0x7fe768e66748>: (1409.2916344265018, 1405.456443360823),
 

In [200]:
# 2014+ rankings
ranking = []
for key in ELO_DICT:
    ranking.append((key, ELO_DICT[key]))

ranking.sort(key=lambda x: x[1], reverse=True)
counter = 1
for i,rank in enumerate(ranking):
    if driver_dict[rank[0]].number != '\\N':
        pos = counter
        fname = driver_dict[rank[0]].firstName
        lname = driver_dict[rank[0]].lastName
        elo = rank[1]
        print(f'{pos}. {fname} {lname} - {elo}')
        counter+=1

1. Lewis Hamilton - 1736.3119435568615
2. Nico Rosberg - 1669.961404686599
3. Max Verstappen - 1597.6238368811835
4. Valtteri Bottas - 1506.2219426612285
5. Carlos Sainz - 1484.013124131847
6. Charles Leclerc - 1432.729986778366
7. Lando Norris - 1431.1119610229591
8. Sergio Pérez - 1412.4182525057997
9. Alexander Albon - 1405.7735131711565
10. Daniel Ricciardo - 1393.0306764159147
11. Pierre Gasly - 1377.6181706732038
12. Felipe Massa - 1325.7454902748925
13. Esteban Ocon - 1323.1257809166743
14. Fernando Alonso - 1313.7761756271047
15. Daniil Kvyat - 1298.0876010099817
16. Sebastian Vettel - 1267.4929641935096
17. Nico Hülkenberg - 1263.5736080835256
18. Lance Stroll - 1242.9782266070918
19. Jean-Éric Vergne - 1225.3817407492058
20. Kimi Räikkönen - 1209.9594498630452
21. Yuki Tsunoda - 1205.568951221904
22. Antonio Giovinazzi - 1201.6068199272477
23. Stoffel Vandoorne - 1199.795584727886
24. Jenson Button - 1196.7181521961384
25. Jack Aitken - 1190.8058966871683
26. Pastor Maldonado